Get the average temporal correlation values of all the spontaneous activities in one exp for 2 cells

In [5]:
# ec_id = 511510650
eid = 501836392 #501886692 three_session_B 501836392 three_session_A 501717543 three_session_C
c1 = 517419703 
c2 = 517419533 

In [6]:
import os
drive_path = '/media/stephen/Brain20191/allen-brain-observatory/visual-coding-2p/'
if not os.path.isdir(drive_path):
    drive_path = '/media/alihamodi/Brain2019/allen-brain-observatory/visual-coding-2p/'
    
import warnings;
warnings.simplefilter('ignore');
import os
import pandas as pd
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import allensdk.brain_observatory.stimulus_info as stim_info
from utils.data_wrappers import *
from utils.polardata import *
from utils.polarplot import *
manifest_file = os.path.join(drive_path,'manifest.json')
boc = BrainObservatoryCache(manifest_file=manifest_file)

In [7]:
from utils.data_wrappers import *
avg_temp_corr_one_exp(boc, eid, c1, c2)

-0.0019509815338686607